In [1]:
import numpy as np
import lmdb
import os
import pickle
import torch
# from mmf.utils.file_io import PathManager
from iopath.common.file_io import PathManager as pm

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import json
PathManager = pm()

In [9]:
# data_path = "/fsx/lyuchen2/vilt_dataset/large_experiments/cmd/cc/validation"

# img = mpimg.imread(os.path.join(data_path, "640130504"))
# plt.imshow(img)
# val_sample = np.load(os.path.join(data_path, "2661647049"), allow_pickle=True)
data_path = "/home/zmykevin/fb_intern/data/mmf_data/datasets/cc/defaults/annotations/train_vinvl_bookcorpus_retrieved_sorted_all_nps_8.npy"
#data_path = "/home/zmykevin/fb_intern/data/mmf_data/datasets/cc/defaults/annotations/train_vinvl_bc_retrieved_6.npy"
annotation = np.load(data_path, allow_pickle=True)
print(len(annotation))

259596


In [10]:
new_annotation = []
with open("/home/zmykevin/fb_intern/data/mingyang_data/CC/cc_objects_id.json", "r") as f:
    object_ids = json.load(f)
for x in tqdm(annotation):
    #print(x)
    y = x.copy()
    current_object_ids = [object_ids[x] for x in y["objects"]]
    y["objects_ids"] = current_object_ids
    new_annotation.append(y)
    #break
#print(new_annotation)
#     for y in  x["objects"]:
#         object_dict[y] = True

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 259596/259596 [00:03<00:00, 83314.37it/s]


In [11]:
with open("/home/zmykevin/fb_intern/data/mmf_data/datasets/cc/defaults/annotations/train_vinvl_bc_retrieved_8.npy", "wb") as f:
    np.save(f, new_annotation)

In [13]:
print(len(object_dict.keys()))
print(object_dict.keys())

NameError: name 'object_dict' is not defined

In [6]:
print(len(val_annotation))

14561


In [4]:
print(val_annotation[0])

{'image_id': 3318333, 'captions': ['author : a life in photography -- in pictures']}


In [5]:
print(type(val_annotation[1]['image_id']))

<class 'int'>


In [2]:
class PaddedFasterRCNNFeatureReader:
    def __init__(self, max_loc):
        self.max_loc = max_loc
        self.first = True
        self.take_item = False

    def _load(self, image_feat_path):
        image_info = {}
        image_info["features"] = load_feat(image_feat_path)

        info_path = "{}_info.npy".format(image_feat_path.split(".npy")[0])
        if PathManager.exists(info_path):
            image_info.update(load_feat(info_path).item())

        return image_info

    def read(self, image_feat_path):
        image_info = self._load(image_feat_path)
        if self.first:
            self.first = False
            if (
                image_info["features"].size == 1
                and "image_feat" in image_info["features"].item()
            ):
                self.take_item = True

        image_feature = image_info["features"]
#         print(image_info["features"].size)
        if self.take_item:
            item = image_info["features"].item()
            if "image_text" in item:
                image_info["image_text"] = item["image_text"]
                image_info["is_ocr"] = item["image_bbox_source"]
                image_feature = item["image_feat"]

            if "info" in item:
                if "image_text" in item["info"]:
                    image_info.update(item["info"])
                image_feature = item["feature"]

        # Handle case of features with class probs
        if (
            image_info["features"].size == 1
            and "features" in image_info["features"].item()
        ):
            item = image_info["features"].item()
            image_feature = item["features"]
            image_info["image_height"] = item["image_height"]
            image_info["image_width"] = item["image_width"]

            # Resize these to self.max_loc
            image_loc, _ = image_feature.shape
            image_info["cls_prob"] = np.zeros(
                (self.max_loc, item["cls_prob"].shape[1]), dtype=np.float32
            )
            image_info["cls_prob"][0:image_loc,] = item["cls_prob"][: self.max_loc, :]
            image_info["bbox"] = np.zeros(
                (self.max_loc, item["bbox"].shape[1]), dtype=np.float32
            )
            image_info["bbox"][0:image_loc,] = item["bbox"][: self.max_loc, :]
            image_info["num_boxes"] = item["num_boxes"]

        # Handle the case of ResNet152 features
        if len(image_feature.shape) > 2:
            shape = image_feature.shape
            image_feature = image_feature.reshape(-1, shape[-1])

        image_loc, image_dim = image_feature.shape
        tmp_image_feat = np.zeros((self.max_loc, image_dim), dtype=np.float32)
        tmp_image_feat[0:image_loc,] = image_feature[: self.max_loc, :]  # noqa
        image_feature = torch.from_numpy(tmp_image_feat)

        del image_info["features"]
        image_info["max_features"] = torch.tensor(image_loc, dtype=torch.long)
        return image_feature, image_info


class LMDBFeatureReader(PaddedFasterRCNNFeatureReader):
    def __init__(self, max_loc, base_path):
        super().__init__(max_loc)
        self.db_path = base_path

        if not PathManager.exists(self.db_path):
            raise RuntimeError(
                "{} path specified for LMDB features doesn't exists.".format(
                    self.db_path
                )
            )
        self.env = None

    def _init_db(self):
        self.env = lmdb.open(
            self.db_path,
            subdir=os.path.isdir(self.db_path),
            readonly=True,
            lock=False,
            readahead=False,
            meminit=False,
        )
        with self.env.begin(write=False, buffers=True) as txn:
            self.image_ids = pickle.loads(txn.get(b"keys"))
            self.image_id_indices = {
                self.image_ids[i]: i for i in range(0, len(self.image_ids))
            }

    def _load(self, image_file_path):
        #print("env is: {}".format(self.env))
        if self.env is None:
            #print("initialize db")
            self._init_db()

        split = os.path.relpath(image_file_path, self.db_path).split(".npy")[0]

        try:
            image_id = int(split.split("_")[-1])
            # Try fetching to see if it actually exists otherwise fall back to
            # default
            img_id_idx = self.image_id_indices[str(image_id).encode()]
        except (ValueError, KeyError):
            # The image id is complex or involves folder, use it directly
            image_id = str(split).encode()
            img_id_idx = self.image_id_indices[image_id]

        with self.env.begin(write=False, buffers=True) as txn:
            image_info = pickle.loads(txn.get(self.image_ids[img_id_idx]))

        return image_info

In [6]:
base_path = "/fsx/zmykevin/data/mmf_data/datasets/cc/defaults/features/lmdbs/cc_vinvl_train_0.lmdb"

In [7]:
#Load the Image Feature
feature_reader = LMDBFeatureReader(30, base_path)

In [8]:
feats, info = feature_reader.read("/fsx/zmykevin/data/mmf_data/datasets/cc/defaults/features/lmdbs/cc_vinvl_train_0.lmdb/cc_0.npy")

KeyError: b'cc_0'

In [28]:
print(feats.size())

torch.Size([30, 2048])


In [24]:
print(info.keys())

dict_keys(['image_id', 'feature_path', 'image_height', 'image_width', 'num_boxes', 'objects', 'bbox', 'cls_prob', 'max_features'])


In [55]:
print(info['image_id'])

0


In [56]:
print(info['feature_path'])

cc_00000000


In [54]:
print(len(info['objects']))

60


In [33]:
print(info['cls_prob'][0].shape)

(1601,)


In [34]:
print(info['max_features'])

tensor(100)


In [29]:
import csv
original_cc_val_path = "/data/home/zmykevin/vinvl_data/CC/Validation_GCC-1.1.0-Validation.tsv"
val = []
with open(original_cc_val_path) as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        val.append(row)
print(val)

['author : a life in photography -- in pictures', 'https://i.pinimg.com/736x/66/01/6c/66016c3ba27c0e04f39e2bd81a934e3e--anita-ekberg-bob-hope.jpg']


In [53]:
#Load the npy file


In [54]:
print(val_vinvl[0])

{'image_id': 3318333, 'captions': ['a blind boy with a piano']}


In [4]:
import json
#load the retrieved caption file
retrieved_data_path = "/data/home/zmykevin/vinvl_data/CC/captions_retrieved_bertembedding_bookcorpus_sorted_top100.json"
with open(retrieved_data_path, "r") as f:
    retrieved_captions = json.load(f)
    
with open("/data/home/zmykevin/vinvl_data/CC/bookcorpus_sentences.json", "r") as f:
    bookcorpus_captions = json.load(f)

with open("/data/home/zmykevin/vinvl_data/CC/cc_objects_captions.json", "r") as f:
    cc_objects_captions = json.load(f)
#print("finish loading the captions")
with open("/fsx/zmykevin/data/mmf_data/datasets/cc/defaults/annotations/cc_objects_id.json", "r") as f:
    object_ids = json.load(f)

In [3]:
print(len(cc_objects_captions))
print(len(bookcorpus_captions))

3116254
14024119


In [18]:
from tqdm import tqdm
index_list = [0,1,2,3,6,7,8,9]

#print(len(retrieved_captions))
for index in index_list:
    data_path = "/fsx/zmykevin/data/mmf_data/datasets/cc/defaults/annotations"
    npy_path = os.path.join(data_path, "train_vinvl_{}.npy".format(index))
    val_vinvl = np.load(npy_path, allow_pickle=True)
    #print(val_vinvl[0])
    sbert_retrieved_vinvl = []
    #for x in val_vinvl:
    #for i in range(0, len(val_vinvl), 5):
    for x in tqdm(val_vinvl):
        #x = val_vinvl[i]
        #print(x)
        current_image_id = x['image_id']
        #print(x["captions"])
        retrieved_caption_ids = retrieved_captions.get(str(current_image_id), None)
        if retrieved_caption_ids is not None:
            retrieved_caption_ids = retrieved_caption_ids[:10]
            #assert len(retrieved_caption_ids) <= 5 and len(retrieved_caption_ids) >= 1
            #current_retrieved_captions = x["captions"]
            #current_retrieved_captions = [cc_objects_captions[i]['caption'] for i in retrieved_caption_ids]
            current_retrieved_captions = [bookcorpus_captions[i]['caption'] for i in retrieved_caption_ids]
            #current_retrieved_caption = cc_objects_captions[retrieved_caption_ids[0]]['caption']
            #current_retrieved_caption = cc_objects_captions[str(current_image_id)]['objects']
            current_object_list = cc_objects_captions[str(current_image_id)]['object_list']
            current_object_ids = [object_ids[x] for x in current_object_list]
            for cr_caption in current_retrieved_captions:
                sbert_retrieved_vinvl.append({'image_id':current_image_id, 'captions':[cr_caption], 'objects':current_object_list, 'objects_ids':current_object_ids})
#         break
#     break
    
    #Save the data to target
    with open(os.path.join(data_path, "train_vinvl_bookcorpus_retrieved_sorted_all_top10_{}.npy".format(index)), "wb") as f:
        print("train_vinvl_bookcorpus_retrieved_sorted_all_top10_{}.npy".format(index))
        np.save(f, sbert_retrieved_vinvl)

100%|██████████| 259957/259957 [02:32<00:00, 1699.96it/s]


train_vinvl_bookcorpus_retrieved_sorted_all_top100_0.npy


100%|██████████| 259795/259795 [02:04<00:00, 2089.86it/s]


train_vinvl_bookcorpus_retrieved_sorted_all_top100_1.npy


100%|██████████| 259835/259835 [01:55<00:00, 2252.51it/s]


...

train_vinvl_bookcorpus_retrieved_sorted_all_top100_7.npy


100%|██████████| 259596/259596 [02:00<00:00, 2159.39it/s]


train_vinvl_bookcorpus_retrieved_sorted_all_top100_8.npy


100%|██████████| 764124/764124 [04:06<00:00, 3094.66it/s] 


train_vinvl_bookcorpus_retrieved_sorted_all_top100_9.npy


In [6]:
with open("/fsx/zmykevin/data/mmf_data/datasets/cc/defaults/annotations/cc_objects_id.json", "r") as f:
    object_ids = json.load(f)
print(object_ids)

{'shirt': 0, 'balcony': 1, 'car': 2, 'van': 3, 'sign': 4, 'bus': 5, 'pant': 6, 'roof': 7, 'boat': 8, 'street': 9, 'road': 10, 'person': 11, 'door': 12, 'window': 13, 'hat': 14, 'building': 15, 'man': 16, 'hair': 17, 'leg': 18, 'picture': 19, 'face': 20, 'top': 21, 'woman': 22, 'poster': 23, 'neck': 24, 'dress': 25, 'arm': 26, 'hand': 27, 'microphone': 28, 'drum': 29, 'skirt': 30, 'wall': 31, 'shoe': 32, 'tripod': 33, 'girl': 34, 'bag': 35, 'scarf': 36, 'rug': 37, 'handle': 38, 'tub': 39, 'fence': 40, 'piano': 41, 'drawer': 42, 'glasses': 43, 'closet': 44, 'clothing': 45, 'cabinet': 46, 'floor': 47, 'jean': 48, 'hanger': 49, 'bathtub': 50, 'clothes': 51, 'pillow': 52, 'tile': 53, 'shadow': 54, 'brick': 55, 'fireplace': 56, 'bar': 57, 'word': 58, 'letter': 59, 'bench': 60, 'ceiling': 61, 'log': 62, 'fire': 63, 'room': 64, 'desk': 65, 'light': 66, 'ledge': 67, 'string': 68, 'sky': 69, 'foot': 70, 'boot': 71, 'handlebar': 72, 'necklace': 73, 'label': 74, 'bottle': 75, 'jacket': 76, 'bracel

In [13]:
data_path = "/fsx/zmykevin/data/mmf_data/datasets/cc/defaults/annotations"
npy_path = os.path.join(data_path, "train_vinvl_0.npy")
val_vinvl = np.load(npy_path, allow_pickle=True)
image_id = val_vinvl[0]['image_id']
#print(val_vinvl[0])
print(len(bookcorpus_captions))
new_val_vinvl = []
for x,c in bookcorpus_captions.items():
    caption = c['caption']
    new_val_vinvl.append({'image_id': image_id, 'captions': [caption]})
    #print(c['caption'])
    #print(new_val_vinvl)
    #break
# new_val_vinvl = []
# for x in val_vinvl:
#     current_image_id = x['image_id']
#     current_captions = x['captions'].copy()
#     current_object_list = cc_objects_captions[str(current_image_id)]['object_list']
#     current_object_ids = [object_ids[x] for x in current_object_list]
#     new_val_vinvl.append({'image_id':current_image_id, 'captions':current_captions, 'objects':current_object_list, 'objects_ids':current_object_ids})
#     #print(new_val_vinvl)
#     #break
print(len(new_val_vinvl))
with open(os.path.join(data_path, "train_vinvl_bookcorpus.npy".format(index)), "wb") as f:
    np.save(f, new_val_vinvl)


14024119
14024119


In [52]:
#Load the error ids
# with open("/data/home/zmykevin/vinvl_data/CC/captions_retrieved_sorted_error_ids.json", "r") as f:
#     retrieved_captions_errors = json.load(f)
# print(len(retrieved_captions_errors))
with open(os.path.join(data_path, "val_vinvl_tfidf_retrieved.npy"), "wb") as f:
    np.save(f, tfidf_retrieved_vinvl)

{'image_id': 0, 'captions': ['man on the street from bus']}


In [4]:
data_path = "/fsx/zmykevin/data/mmf_data/datasets/cc/defaults/annotations"
index_list = [0,1,2,3,6,7,8,9]

for chunk_id in index_list:
    npy_path = os.path.join(data_path, "train_vinvl_bookcorpus_retrieved_sorted_all_nps_cs{}.npy".format(chunk_id))
    #npy_path = os.path.join(data_path, "val_vinvl_itm.npy")
    val_vinvl = np.load(npy_path, allow_pickle=True)
    #print(val_vinvl[0])
    new_val_vinvl = []
    for x in tqdm(val_vinvl):
        new_x = x.copy()
        #new_x.pop("object_ids")
        noun_phrases = new_x.pop("noun_phrases")
        noun_phrases_text = []
        num_candidates = len(noun_phrases)
        objects_norep = new_x.pop("objects_norep")
        new_objects_norep = {ob_k:[] for ob_k in objects_norep}
        for c_np in noun_phrases:
            current_np_text = [list(y.keys())[0] for y in c_np]
            noun_phrases_text.append(current_np_text)
            #get the top scores for each objects
            try:
                current_scores = np.concatenate([np.expand_dims(list(y.values())[0],axis=0) for y in c_np])
                current_max_scores = np.max(current_scores,axis=0)
                current_max_scores = (current_max_scores/0.5) * (current_max_scores > 0.5)
                current_max_args = np.argmax(current_scores,axis=0)
            except:
                current_max_scores = np.zeros(len(objects_norep),dtype=np.float32)
                current_max_args = np.zeros(len(objects_norep))
            
            for i, ob_k in enumerate(objects_norep):
                new_objects_norep[ob_k].append((current_max_scores[i],current_max_args[i]))


            
        new_x["objects_norep"] = new_objects_norep
        new_x["noun_phrases"] = noun_phrases_text
        new_val_vinvl.append(new_x)
#         print(new_val_vinvl)
#         break
#     break
        
    output_path = os.path.join(data_path, "train_vinvl_sbert_bc_nps_fixed_{}.npy".format(chunk_id))
    with open(output_path, "wb") as f:
        print("write out to {}".format(output_path.split("/")[-1]))
        np.save(f, new_val_vinvl)

#Implementation
 

100%|██████████| 258858/258858 [01:50<00:00, 2345.40it/s]


write out to train_vinvl_sbert_bc_nps_fixed_0.npy


100%|██████████| 258663/258663 [01:32<00:00, 2797.37it/s]


write out to train_vinvl_sbert_bc_nps_fixed_1.npy


100%|██████████| 258681/258681 [01:30<00:00, 2849.19it/s]


...

write out to train_vinvl_sbert_bc_nps_fixed_7.npy


100%|██████████| 258457/258457 [01:22<00:00, 3122.64it/s]


write out to train_vinvl_sbert_bc_nps_fixed_8.npy


100%|██████████| 572994/572994 [03:08<00:00, 3042.27it/s]


write out to train_vinvl_sbert_bc_nps_fixed_9.npy


In [5]:
from tqdm import tqdm

output_path = os.path.join(data_path, "train_vinvl_sbert_bc_nps_cs_mini_fixed.npy")
with open(output_path, "wb") as f:
    print("write out to {}".format(output_path.split("/")[-1]))
    np.save(f, new_val_vinvl[:1000])

write out to train_vinvl_sbert_bc_nps_cs_mini_fixed.npy


In [6]:
print(new_val_vinvl[0])

{'image_id': 2500650, 'captions': ["one dude i saw on a poster in my daughter's room had a sweatband on his bald head and an adhesive on his face!", 'making his way over the plush carpets of the main lobby, dodging the scurrying hotel workers, truthful moved past the service bar outside the grand ballroom, next to which was a sign reading: state of the union reception.', 'a strange looking short man with a beard, a top hat, in a full tuxedo suit is sitting on the far side of the room behind a foldable table.', 'when i looked down the hallway of the ground floor, i saw a man wearing a brown leather jacket along with a visor cap slowly approaching me.', 'dressed in balmain moto jeans, a white v-neck t-shirt, black and grey giuseppe zanotti high-top sneakers, and a grey color block lanvin button down cardigan, \xa0this man had my heart palpitating before he could even speak a word.'], 'objects': ['handle', 'handle', 'handle', 'background', 'door handle', 'word', 'letter', 'background', 't